In [1]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 9, 1, Finished, Available)

In [2]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("Suspeito", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 9, 2, Finished, Available)

In [3]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Suspeito.csv')

StatementMeta(tcccibele, 9, 3, Finished, Available)

In [4]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 9, 4, Finished, Available)

In [5]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'Suspeito':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 9, 5, Finished, Available)

In [6]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/Suspeito/')

StatementMeta(tcccibele, 9, 6, Finished, Available)

In [7]:
df = spark.read.parquet('/history_zone/2019/Suspeito/')
df.toPandas()

StatementMeta(tcccibele, 9, 7, Finished, Available)

,Suspeito,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Amante,0,0,0,0,0,0,0,0,0,0,0,0
1,Amigo(a),78,59,62,57,42,39,36,56,35,44,33,17
2,Avó,11,15,4,8,11,4,3,3,8,2,3,4
3,Avô,3,3,1,5,3,4,2,5,3,6,4,3
4,Bisneto(a),2,9,3,1,0,1,1,1,0,0,1,1
5,Chefa,0,0,0,0,0,0,0,0,0,0,0,0
6,Chefe,0,0,7,15,10,11,8,16,11,15,12,6
7,Colega de trabalho,0,0,0,0,0,0,0,0,0,0,0,0
8,Companheiro (a),2746,2191,2296,2223,2220,1863,2008,2317,2158,2327,1998,999
9,Conhecido (a),0,0,0,0,0,0,0,0,0,0,0,0
